In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import tensorflow as tf
Maestro_Model_path = '/content/drive/My Drive/music_transformer_model_2.keras'
Maestro_Model = tf.keras.models.load_model(Maestro_Model_path)

In [7]:
Maestro_Model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 100, 256)       │         75,264 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 100, 256)       │              0 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention      │ (None, 100, 256)       │      2,103,552 │ add[0][0], add[0][0]   │
│ (MultiHeadAttention)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization       │ (None, 100, 256)       │            512 │ multi_head_attention[… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 100, 512)       │        131,584 │ layer_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_1     │ (None, 100, 512)       │          1,024 │ dense[0][0]            │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_1    │ (None, 100, 512)       │      4,200,960 │ layer_normalization_1… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_2     │ (None, 100, 512)       │          1,024 │ multi_head_attention_… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 100, 512)       │        262,656 │ layer_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_3     │ (None, 100, 512)       │          1,024 │ dense_1[0][0]          │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ multi_head_attention_2    │ (None, 100, 512)       │      4,200,960 │ layer_normalization_3… │
│ (MultiHeadAttention)      │                        │                │ layer_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_4     │ (None, 100, 512)       │          1,024 │ multi_head_attention_… │
│ (LayerNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 100, 512)       │        262,656 │ layer_normalization_4… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ layer_normalization_5     │ (None, 100, 512)       │          1,024 │ dense_2[0][0]          │
│ (LayerNormalization)      │                        │                │                        │
├──────────────────────

 Total params: 47,579,252 (181.50 MB)

 Trainable params: 15,859,750 (60.50 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 31,719,502 (121.00 MB)

In [9]:
!pip install midiutil

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for midiutil: filename=MIDIUtil-1.2.1-py3-none-any.whl size=54569 sha256=64ee8dc2c9e22634ed2b94ecbe3c39df03d56fd2e9258f05afd1f568ecea2c89
  Stored in directory: /root/.cache/pip/wheels/af/43/4a/00b5e4f2fe5e2cd6e92b461995a3a97a2cebb30ab5783501b0
Successfully built midiutil


In [14]:
import numpy as np
from midiutil import MIDIFile
import os
import tensorflow as tf

def generate_and_save_diverse_music_tt2(
    model,
    initial_sequences,
    num_pieces,
    length,
    top_k=10,
    temperature=1,
    exclude_top=1,
    output_dir="generated_music_diverse"
):
    os.makedirs(output_dir, exist_ok=True)

    for i in range(num_pieces):
        # Start with a unique seed sequence
        seed_sequence = initial_sequences[i % len(initial_sequences)]
        generated_sequence = seed_sequence.copy()

        for _ in range(length):
            # Model prediction
            input_sequence = np.array([generated_sequence[-len(seed_sequence):]])  # Ensure the sequence length is correct
            predictions = model.predict(input_sequence, verbose=0)[0, -1]  # Get the predicted probabilities for the next token

            # Apply temperature scaling
            scaled_predictions = np.log(predictions + 1e-8) / temperature
            probabilities = np.exp(scaled_predictions)
            probabilities /= np.sum(probabilities)

            # Exclude top-N predictions
            sorted_indices = np.argsort(probabilities)[::-1]
            allowed_indices = sorted_indices[exclude_top:]
            adjusted_probabilities = probabilities[allowed_indices]
            adjusted_probabilities /= np.sum(adjusted_probabilities)

            # Sample from adjusted probabilities
            next_note = np.random.choice(allowed_indices, p=adjusted_probabilities)

            # Append to the generated sequence
            generated_sequence.append(next_note)

        # Save the generated sequence as a MIDI file
        midi_file = MIDIFile(1)
        midi_file.addTempo(0, 0, 120)

        time = 0
        for pitch in generated_sequence:
            if 0 <= pitch <= 127:  # Valid MIDI note range
                midi_file.addNote(0, 0, pitch, time, 1, 100)
                time += 1

        output_path = os.path.join(output_dir, f"Generated_Piece_{i + 1}.mid")
        with open(output_path, "wb") as f:
            midi_file.writeFile(f)

    print(f"Generated {num_pieces} diverse pieces. Check the '{output_dir}' folder.")

initial_sequences = [np.random.randint(0, 307, size=100).tolist() for _ in range(5)]

generate_and_save_diverse_music_tt2(
    model=Maestro_Model,
    initial_sequences=initial_sequences,
    num_pieces=2,
    length=50,
    top_k=20,
    temperature=1.4,
    exclude_top=5,
    output_dir="test_generated_music_from_trained_model"
)

print("Test generation completed. Check the 'test_generated_music_from_trained_model' folder for MIDI files.")


Generated 2 diverse pieces. Check the 'test_generated_music_from_trained_model' folder.
Test generation completed. Check the 'test_generated_music_from_trained_model' folder for MIDI files.
